In [ ]:
!pip install tenseal

In [ ]:
import tenseal as ts

context=ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
context.generate_galois_keys()
context.global_scale=2**40

# context=ts.context(
#         ts.SCHEME_TYPE.CKKS,
#         poly_modulus_degree=32768,-1,
#         coeff_mod_bit_sizes=[60, 50, 40, 40, 40, 50, 60]
#     )
# context.generate_galois_keys()
# context.global_scale=2 ** 25

# poly_mod_degree = 8192
# coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
# # create TenSEALContext
# context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, coeff_mod_bit_sizes)
# context.global_scale = 2 ** 15
# context.generate_galois_keys()

In [ ]:
import numpy as np
import time
import psutil
import matplotlib.pyplot as plt
from scipy.special import expit
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from threading import Thread, Lock

In [ ]:
X_train = pd.read_csv('X_train3.csv')
X_train=X_train.to_numpy()
y_train = pd.read_csv('y_train3.csv')
y_train=y_train.to_numpy().reshape(-1)
X_test = pd.read_csv('X_test3.csv')
X_test=X_test.to_numpy()
y_test = pd.read_csv('y_test3.csv')
y_test=y_test.to_numpy().reshape(-1)

y_train = np.where(y_train == 0, -1, 1)
y_test = np.where(y_test == 0, -1, 1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# X_train = np.round(X_train, 6)

# Lock for controlling CPU monitoring
monitor_lock = Lock()

In [ ]:
enc_X_train=[]
for i in X_train:
  p=ts.ckks_vector(context,i)
  enc_X_train.append(p)

In [ ]:
enc_y_train=[]
for i in y_train:
  p=ts.ckks_vector(context,[i])
  enc_y_train.append(p)

In [ ]:
def relinearlizing_vector(vec):
  vec_2=[]
  vec=vec.decrypt()
  for i in vec:
    p=round(i,7)
    vec_2.append(p)
  vec_2=ts.ckks_vector(context,vec_2)
  return vec_2

In [ ]:
def relinearlizing_scalar(sca):
  sca=sca.decrypt()
  sca=round(sca[0],7)
  sca=ts.ckks_vector(context,[sca])
  return sca

In [ ]:
weights_enc= np.zeros(X_train.shape[1])
bias_enc=0
bias_enc=ts.ckks_vector(context,[bias_enc])
lr=0.001
lambda_param=0.01
n_samples, n_features = X_train.shape
print(n_samples, n_features)

1080 22


In [ ]:
weights_enc=ts.ckks_vector(context,weights_enc)

In [ ]:
def marginal_decrypt(margin):
  margin_x=None
  with monitor_lock:
    margin_x=margin.decrypt()[0]
  if margin_x>=1:
    return True
  else:
    return False

In [ ]:
def get_cpu_and_ram_utilization(pid, duration):
    process = psutil.Process(pid)
    cpu_usages = []
    ram_usages = []
    start_time = time.time()
    while time.time() - start_time < duration:
        # with monitor_lock:
        cpu_usage = process.cpu_percent(interval=1)
        ram_usage = process.memory_percent()
        cpu_usages.append(cpu_usage)
        ram_usages.append(ram_usage)
        print(f"CPU utilization: {cpu_usage}% | RAM utilization: {ram_usage}%")
    total_cpu_usage = sum(cpu_usages)
    return total_cpu_usage, cpu_usages, ram_usages

In [ ]:
pid = os.getpid()

# Duration to monitor CPU usage
monitor_duration = 180

In [ ]:
time.sleep(15)

In [ ]:
results={}
def fit_svm(enc_X_train,enc_y_train,weights_enc,bias_enc,lr=0.001,lambda_param=0.01,epochs=2):
  train_time_start=time.time()
  for epoch in range(epochs):
    for idx, x_i in enumerate(enc_X_train):
      y_i=enc_y_train[idx]

      # condition = y_[idx] * (np.dot(x_i, w) - b) >= 1
      condition=y_i*(x_i.dot(weights_enc)-bias_enc)

      cc=marginal_decrypt(condition)

      r=cc

      if r:
        # print('r executed')
        mon=lr*(2*lambda_param*weights_enc)
        weights_enc2=weights_enc-mon
        weights_enc=weights_enc2
      else:
        # print('else executed')
        r1=2*lambda_param*weights_enc
        # print(f'r1={r1.decrypt()}')
        r2=x_i*y_i
        r2=relinearlizing_vector(r2)
        # print(f'r2={r2.decrypt()}')
        r3=r1-r2
        # print(f'r3={r3.decrypt()}')
        r4=lr*r3
        # print(f'r4={r4.decrypt()}')
        weights_enc2=weights_enc-r4
        bias_enc=bias_enc-lr*y_i
        # print(f'weights_enc2={weights_enc2.decrypt()}')
        weights_enc=weights_enc2

      weights_enc=relinearlizing_vector(weights_enc)
      bias_enc=relinearlizing_scalar(bias_enc)
      # print(f"epoch={epoch},iter={idx} executed")

  results['weights_enc']=weights_enc
  results['bias_enc']=bias_enc
  train_time_end=time.time()
  results['train_time']=train_time_end-train_time_start
  print(results['train_time'])


# results = {}

# def fit_svm(enc_X_train, enc_y_train, weights_enc, bias_enc, lr=0.001, lambda_param=0.01, epochs=2):
#     train_time_start = time.time()

#     for epoch in range(epochs):
#         for idx, x_i in enumerate(enc_X_train):
#             y_i = enc_y_train[idx]

#             # condition = y_[idx] * (np.dot(x_i, w) - b) >= 1
#             condition = y_i * (x_i.dot(weights_enc) - bias_enc)

#             if marginal_decrypt(condition):  # Directly check condition
#                 # print('r executed')
#                 weights_enc = weights_enc - (lr * (2 * lambda_param * weights_enc))  # Expanded -= operator
#             else:
#                 # print('else executed')
#                 # Combine operations to reduce temporary variables and operations
#                 grad_w = 2 * lambda_param * weights_enc - x_i * y_i
#                 weights_enc = weights_enc - (lr * grad_w)  # Expanded -= operator
#                 bias_enc = bias_enc - (lr * y_i)  # Expanded -= operator
#                 print(f'weights_enc={weights_enc.decrypt()}')

#             print(f"epoch={epoch}, iter={idx} executed")

#     # Store results
#     results['weights_enc'] = weights_enc
#     results['bias_enc'] = bias_enc
#     results['train_time'] = time.time() - train_time_start
#     print(results['train_time'])



In [ ]:
train_thread = Thread(target=fit_svm, args=(enc_X_train, enc_y_train, weights_enc, bias_enc))
train_thread.start()

total_cpu_usage, cpu_usages, ram_usages = get_cpu_and_ram_utilization(pid, monitor_duration)
train_thread.join()

print(f"Total CPU utilization over {monitor_duration} seconds: {total_cpu_usage}%")

# Print the CPU and RAM usage per second
print("CPU usage per second:", cpu_usages)
print("RAM usage per second:", ram_usages)

# Calculate total CPU resource consumption in 'CPU-seconds'
cpu_seconds = sum(cpu_usages) / 100
print(f"Total CPU resource consumption: {cpu_seconds} CPU-seconds")

CPU utilization: 101.2% | RAM utilization: 11.445954466587311%
CPU utilization: 100.9% | RAM utilization: 11.445954466587311%
CPU utilization: 99.0% | RAM utilization: 11.445954466587311%
CPU utilization: 96.5% | RAM utilization: 11.445954466587311%
CPU utilization: 97.5% | RAM utilization: 11.445954466587311%
CPU utilization: 97.1% | RAM utilization: 11.445954466587311%
CPU utilization: 100.7% | RAM utilization: 11.445954466587311%
CPU utilization: 103.8% | RAM utilization: 11.445954466587311%
CPU utilization: 102.8% | RAM utilization: 11.445954466587311%
CPU utilization: 100.4% | RAM utilization: 11.445954466587311%
CPU utilization: 102.2% | RAM utilization: 11.445954466587311%
CPU utilization: 102.8% | RAM utilization: 11.445954466587311%
CPU utilization: 101.0% | RAM utilization: 11.445954466587311%
CPU utilization: 103.9% | RAM utilization: 11.445954466587311%
CPU utilization: 97.9% | RAM utilization: 11.445954466587311%
CPU utilization: 105.3% | RAM utilization: 11.44595446658731

In [ ]:
weights_unenc=results['weights_enc'].decrypt()
print(weights_unenc)
bias_unenc=results['bias_enc'].decrypt()
print(bias_unenc)

[0.0587476005220892, 0.211065100381582, -0.024944700931275557, 0.17217220192252503, -0.10887489838921245, 0.0237975004635364, -0.00972740174752676, 0.02265250003305329, 0.3464674009141069, 0.11069179916412838, 0.21363759988467235, 0.23628279934052376, 0.13906379936833085, 0.10515780246815191, 0.15033850043428854, 0.10675980006803527, 0.10893879993336596, -0.059158701237083736, -0.0743538012260886, -0.21179830010196282, -0.28071170207610135, -0.08063389872616024]
[-0.06899999960374968]


In [ ]:
enc_X_test=[]
for i in X_test:
  p=ts.ckks_vector(context,i)
  enc_X_test.append(p)

In [ ]:
def test_edge_service(appx):
  appx=appx.decrypt()[0]
  return np.sign(appx)

In [ ]:
def predict(enc_X_test,weights_enc,bias_enc):
  unenc_preds=[]
  for i in enc_X_test:
    appx=(i.dot(weights_enc))-bias_enc
    appx=test_edge_service(appx)
    unenc_preds.append(appx)
  return unenc_preds

In [ ]:
test_time_start=time.time()
y_pred=predict(enc_X_test,results['weights_enc'],results['bias_enc'])
test_time=time.time()-test_time_start
results['test_time']=test_time
print(results['test_time'])

11.493041515350342


In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.8333333333333334

In [ ]:
import joblib
joblib.dump((results['weights_enc'].decrypt(), results['bias_enc'].decrypt()[0], accuracy, y_pred, cpu_seconds, cpu_usages, ram_usages, results['train_time'], results['test_time']), 'variables_ckks.pkl')

['variables_ckks.pkl']